In [7]:
from nba_api.stats.static import players,teams
from nba_api.stats.endpoints import commonteamroster,shotchartdetail,boxscoreadvancedv2,boxscoreplayertrackv2,playergamelog,boxscorematchups,leaguegamefinder,playbyplayv2
import json
import pandas as pd
from pandas.io.json import json_normalize
import time

custom_headers = {
        'Host': 'stats.nba.com',
        'Connection': 'keep-alive',
        'Accept': 'application/json, text/plain, */*',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
        'Referer': 'https://stats.nba.com/',
        "x-nba-stats-origin": "stats",
        "x-nba-stats-token": "true",
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        }

def getroster(i):
    roster = pd.DataFrame(json.loads(commonteamroster.CommonTeamRoster(team_id=i,season='2019-20',headers=custom_headers,timeout=30).get_json())['resultSets'][0]['rowSet'],
                                   columns=json.loads(commonteamroster.CommonTeamRoster(team_id=i,season='2019-20',headers=custom_headers,
                                                                                timeout=30).get_json())['resultSets'][0]['headers'])
    return roster

def getshots(i,t):
    shots = pd.DataFrame(json.loads(shotchartdetail.ShotChartDetail(player_id=i,context_measure_simple='FGA',
                                                        season_nullable='2019-20',
                                                        team_id=t,headers=custom_headers,timeout=30).get_json())['resultSets'][0]['rowSet'],
                                     columns=json.loads(shotchartdetail.ShotChartDetail(player_id=i,context_measure_simple='FGA',
                                                        season_nullable='2019-20',
                                                        team_id=t,headers=custom_headers,timeout=30).get_json())['resultSets'][0]['headers'])
    return shots

def gettouches(g):
    touches = pd.DataFrame(json.loads(boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id=g,headers=custom_headers,timeout=1800).get_json())['resultSets'][0]['rowSet'],
            columns=json.loads(boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id=g,headers=custom_headers,timeout=1800).get_json())['resultSets'][0]['headers'])
    return touches

def getboxscore(p):
    box = pd.DataFrame(json.loads(playergamelog.PlayerGameLog(player_id=p,season='2019-20',headers=custom_headers,timeout=1800).get_json())['resultSets'][0]['rowSet'],
            columns=json.loads(playergamelog.PlayerGameLog(player_id=p,season='2019-20',headers=custom_headers,timeout=1800).get_json())['resultSets'][0]['headers'])
    return box
    
def getmatchups(g):
    matchup = pd.DataFrame(json.loads(boxscorematchups.BoxScoreMatchups(game_id=g,headers=custom_headers,timeout=1800).get_json())['resultSets'][0]['rowSet'],
            columns=json.loads(boxscorematchups.BoxScoreMatchups(game_id=g,headers=custom_headers,timeout=1800).get_json())['resultSets'][0]['headers'])
    return matchup

In [8]:
######### Gets roster player ids for all teams #############
team_ids = json_normalize(teams.get_teams())['id'].unique().tolist()
team_names = json_normalize(teams.get_teams())[['id','full_name','abbreviation']]
players_full_list = pd.DataFrame()

for i in team_ids:
    result = getroster(i)
    players_full_list = pd.concat([players_full_list,result])
    time.sleep(2)

joined_rosters = pd.merge(players_full_list,team_names,how='inner',left_on='TeamID',right_on='id')
joined_rosters.head(10)

In [9]:
############### Gets all shot attempts/locations from team roster id list ##############
players = joined_rosters.groupby(['PLAYER_ID','id']).apply(list).reset_index(name='new')
player_ids = players['PLAYER_ID'].tolist()
team_ids_playerorder = players['id'].tolist()
shot_full_list = pd.DataFrame()

for i,t in zip(player_ids,team_ids_playerorder):
    try:
        shot_response = getshots(i,t)
    except ValueError:
        continue
    else: 
        shot_full_list = pd.concat([shot_full_list,shot_response])
        time.sleep(2)
shot_full_list = pd.merge(shot_full_list,team_names,how='left',left_on='TEAM_ID',right_on='id')
shot_full_list = pd.merge(shot_full_list,players_full_list,how='left',left_on='PLAYER_ID',right_on='PLAYER_ID')
shot_full_list.head(10)
# shot_full_list.to_csv('shotchartdetail.csv')

In [10]:
game_ids = pd.DataFrame(json.loads(leaguegamefinder.LeagueGameFinder(season_nullable='2019-20',league_id_nullable='00',
                                                          season_type_nullable='Regular Season',headers=custom_headers,
                                                                     timeout=1800).get_json())['resultSets'][0]['rowSet'],
            columns=json.loads(leaguegamefinder.LeagueGameFinder(season_nullable='2019-20',league_id_nullable='00',
                                                                 season_type_nullable='Regular Season',headers=custom_headers,
                                                                 timeout=1800).get_json())['resultSets'][0]['headers'])['GAME_ID'].unique().tolist()
# ########## Player touch points ######################

touchpoints_list = pd.DataFrame()

for g in game_ids:
    try:
        touch_response = gettouches(g)
    except ValueError:
        continue
    else: 
        touchpoints_list = pd.concat([touchpoints_list,touch_response])
        time.sleep(2)

touchpoints_list['new_min'] = touchpoints_list['MIN'].str.split(':',n=1,expand=True)[0]
touchpoints_list['new_sec'] = touchpoints_list['MIN'].str.split(':',n=1,expand=True)[1]
touchpoints_list.head(10)
# touchpoints_list.to_csv('touchesdetail.csv')

In [12]:
########## Player game by game stats #################
game_boxes_list = pd.DataFrame()

for p in player_ids:
    try:
        box = getboxscore(p)
    except ValueError:
        continue
    else:
        game_boxes_list = pd.concat([game_boxes_list,box])
        time.sleep(2)

game_boxes_list.head(10)
# game_boxes_list.to_csv('boxscores.csv')

In [17]:
######### Player game matchup players ################
matchup_list = pd.DataFrame()

for g in game_ids:
    try:
        matchup = getmatchups(g)
    except ValueError:
        continue
    else:
        matchup_list = pd.concat([matchup_list,matchup])
        time.sleep(2)

matchup_list['new_min'] = matchup_list['MATCHUP_MIN'].str.split(':',n=1,expand=True)[0]
matchup_list['new_sec'] = matchup_list['MATCHUP_MIN'].str.split(':',n=1,expand=True)[1]
matchup_list.head(10)
# matchup_list.to_csv('matchup.csv')